In [1]:
import pandas as pd

orders = pd.read_csv("orders_clean - Sheet1.csv")
delivery = pd.read_csv("delivery_clean - Sheet1.csv")
inventory = pd.read_csv("Inventry_clean - Sheet1.csv")

print("ORDERS columns:", list(orders.columns))
print("DELIVERY columns:", list(delivery.columns))
print("INVENTORY columns:", list(inventory.columns))

orders.head(3), delivery.head(3), inventory.head(3)


ORDERS columns: ['CustomerID', 'Order Timestamp', 'AOV', 'CustomerRating', 'Feedback', 'Unnamed: 5', 'Average Customer Rating', 'Average AOV', 'Unnamed: 8', 'Month ', 'Average AOV.1', 'Average Customer Rating.1']
DELIVERY columns: ['OrderID', 'RiderID', 'Time To Dispatch', 'Time In Transit', 'Delivery LocationPincode', 'Delivery Status ', 'Unnamed: 6', 'Average dispatch time ', 'Average transit time ', 'Unnamed: 9', 'Deliveries ', 'No. of deliveries']
INVENTORY columns: ['Product Category', 'Week', 'Stockout Incidents', 'Unnamed: 3', 'Category ', 'Total ', 'Total stockout incidents', 'Unnamed: 7', 'Week ', 'Total stockouts ', 'Unnamed: 10', 'Week .1', 'Average stockout  incidents', 'Round ']


(  CustomerID   Order Timestamp      AOV  CustomerRating  \
 0  CUST_0120  2/1/2025 5:14:00  ₹479.94               3   
 1  CUST_0001  2/1/2025 7:22:00  ₹638.46               5   
 2  CUST_0090  2/1/2025 7:43:00  ₹425.61               5   
 
                       Feedback  Unnamed: 5  Average Customer Rating  \
 0       Good quality products.         NaN                    3.663   
 1         Loved the discounts.         NaN                      NaN   
 2  Product quality acceptable.         NaN                      NaN   
 
   Average AOV  Unnamed: 8  Month  Average AOV.1  Average Customer Rating.1  
 0     ₹407.50         NaN     2.0       ₹415.13                   3.569536  
 1         NaN         NaN     3.0       ₹409.15                   3.638418  
 2         NaN         NaN     4.0       ₹416.06                   3.678161  ,
     OrderID  RiderID  Time To Dispatch  Time In Transit  \
 0  ORD_0229  RID_008                 5               20   
 1  ORD_0208  RID_006              

In [2]:

import numpy as np

# === Parameters you can adjust ===


def find_col(candidates, columns):
    cols_low = {c.lower(): c for c in columns}
    for cand in candidates:
        for col in columns:
            if cand in col.lower():
                return col
    return None

# ORDERS mapping
order_ts_col = find_col(['order timestamp','order_date','order time','timestamp'], orders.columns)
customer_id_col = find_col(['customerid','customer id','cust_id'], orders.columns)
revenue_col = find_col(['revenue','amount','order value','aov','total'], orders.columns)
promo_col = find_col(['promo','promotion','coupon','offer'], orders.columns)
month_col = find_col(['month'], orders.columns)

# DELIVERY mapping
transit_col = find_col(['transit time','delivery time','time to deliver','time to delivery','transit'], delivery.columns)
dispatch_col = find_col(['time to dispatch','dispatch'], delivery.columns)
slot_col = find_col(['slot','time slot','delivery slot'], delivery.columns)
zone_col = find_col(['zone','area','region'], delivery.columns)
distance_col = find_col(['distance'], delivery.columns)
late_col = find_col(['late','delay','over sla'], delivery.columns)

# INVENTORY mapping
category_col = find_col(['product category','category'], inventory.columns)
week_col = find_col(['week'], inventory.columns)
stockout_col = find_col(['stockout incidents','stock out','stockout','oos'], inventory.columns)

mapping = {
    "orders": {
        "timestamp": order_ts_col,
        "customer_id": customer_id_col,
        "revenue": revenue_col,
        "promo_flag": promo_col,
        "month": month_col,
    },
    "delivery": {
        "transit_time_mins": transit_col,
        "dispatch_time_mins": dispatch_col,
        "slot": slot_col,
        "zone": zone_col,
        "distance_km": distance_col,
        "late_flag": late_col,
    },
    "inventory": {
        "product_category": category_col,
        "week": week_col,
        "stockout_incidents": stockout_col,
    }
}

mapping


{'orders': {'timestamp': 'Order Timestamp',
  'customer_id': 'CustomerID',
  'revenue': 'AOV',
  'promo_flag': None,
  'month': 'Month '},
 'delivery': {'transit_time_mins': 'Average transit time ',
  'dispatch_time_mins': 'Time To Dispatch',
  'slot': None,
  'zone': None,
  'distance_km': None,
  'late_flag': None},
 'inventory': {'product_category': 'Product Category',
  'week': 'Week',
  'stockout_incidents': 'Stockout Incidents'}}

In [4]:

from scipy import stats
ttest_output = "Skipped (promo/coupon column not found)."

rev = mapping["orders"]["revenue"]
promo = mapping["orders"]["promo_flag"]

if rev and promo and rev in orders.columns and promo in orders.columns:

    promo_series = orders[promo].astype(str).str.strip().str.lower().isin(['1','true','yes','y','promo','applied'])
    a = pd.to_numeric(orders.loc[promo_series, rev], errors='coerce').dropna()
    b = pd.to_numeric(orders.loc[~promo_series, rev], errors='coerce').dropna()
    if len(a) > 2 and len(b) > 2:
        ttest_output = stats.ttest_ind(a, b, equal_var=False)
ttest_output


'Skipped (promo/coupon column not found).'

## 3) Chi-Square — Late Deliveries vs Slot

In [5]:

chi_output = "Skipped (slot or late_flag not found)."
slot = mapping["delivery"]["slot"]
late = mapping["delivery"]["late_flag"]

if slot and late and slot in delivery.columns and late in delivery.columns:
    table = pd.crosstab(delivery[slot], delivery[late])
    if table.shape[0] > 1 and table.shape[1] > 1:
        chi2, p, dof, expected = stats.chi2_contingency(table)
        chi_output = {"chi2": chi2, "p": p, "dof": dof}
chi_output


'Skipped (slot or late_flag not found).'